In [1]:
from blablacarapi import BlaBlaCarApi
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, date

In [8]:
# initialize API
api = BlaBlaCarApi(api_key="e359fe6292be4078ba1dcc963577813f", locale = 'it_IT')

In [4]:
def scraping(url):
    RE_PUB_DATE = re.compile(r'.* (\d+\/\d+\/\d+)')
    RE_VIEWS = re.compile(r'.* (\d+) .*')
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    #print soup.prettify()
    info = soup.find('div', attrs={'class': 'RideDetails-publicationInfo u-table'}).text #Find (at most) *one*
    info = info.replace('\n','').replace('\t','').split('-')
    pub_date_str = info[0].strip()
    pub_date = RE_PUB_DATE.match(pub_date_str).groups()[0]
    views_str = info[1].strip()

    pub_date = datetime.strptime(pub_date, "%d/%m/%Y").date()  # .date() because we just need the day and not also the time
                                                                # this is the same format as depdate
    views = int(RE_VIEWS.match(views_str).groups()[0])
    return pub_date, views

In [ ]:
def get_data_API(year,month,day_begin,day_end):
    yearmonth = str(year)+'-'+str(month).zfill(2)+'-'
    trip_data = []
    for day in xrange(day_begin, day_end+1):
        page = 1
        date_query = yearmonth + str(day).zfill(2)
        trips = api.trips(frm="Milano", locale='it_IT', limit=100, db = date_query, de = date_query)
        for t in trips.trips:
            trip_data.append(extract_data(t))
            
            while trips.pager.has_next():
                page += 1
                trips = api.trips(frm="Milano", page = page, locale='it_IT', limit=100, db = date_query, de = date_query)
                for t in trips.trips:
                    trip_data.append(extract_data(t))
    
    return tuplelist_to_df(trip_data)

In [5]:
def extract_data(trip):
    depdate = trip.departure_date
    depplace = trip.departure_place
    arrplace = trip.arrival_place
    
    link = trip.links['_front']    # link to the page of the trip (available only for trips "in the future")
    pub_date, views = scraping(link)
    
    dep_day = depdate.date()       # date of the trip
    dep_time = depdate.time()      # hour of departure
    dep_city = depplace['city_name']  # city of dep
    dep_lat = depplace['latitude']    # latitude of dep place
    dep_long = depplace['longitude']  # longitude of dep place
    dep_addr = depplace['address']    # address of dep place
    arr_city = arrplace['city_name']  
    arr_lat = arrplace['latitude']
    arr_long = arrplace['longitude']
    arr_addr = arrplace['address']
    price = trip.price_with_commission['value'] # price with commission
    seatsleft = trip.seats_left                 # seats left 
    seats = trip.seats                          # seats made available by the driver
    trip_id = trip.permanent_id                 # trip id
    stops = trip.locations_to_display           # locations in which the driver will stop
    bookmode = trip.booking_mode                # could be MANUAL if the driver has to accept your request, AUTO otherwise
    booktype = trip.booking_type                # payment method: ONLINE, ON BOARD
    
    return (link, dep_day, dep_time, dep_city, dep_lat, dep_long, dep_addr, arr_city, arr_lat, arr_long, arr_addr, price, 
         seatsleft, seats, trip_id, stops, bookmode, booktype, pub_date, views, date.today())

In [6]:
def tuplelist_to_df(L):
    labels = ['link', 'dep_day', 'dep_time', 'dep_city', 'dep_lat', 'dep_long', 'dep_addr', 'arr_city', 'arr_lat', 'arr_long', 
              'arr_addr', 'price', 'seatsleft', 'seats', 'trip_id', 'stops', 'bookmode', 'booktype','pub_date','views','today']
    df = pd.DataFrame.from_records(L, columns = labels)
    return df

In [9]:
df_mar = get_data_API(2017,3,30,31)
print len(df_mar)

/opt/conda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


626


In [10]:
df_mar.to_csv('./Data/Future/March2017.csv', encoding = 'utf-8')

In [11]:
df_apr = get_data_API(2017,4,1,30)
print len(df_apr)

267


In [12]:
df_apr.to_csv('./Data/Future/April2017.csv', encoding = 'utf-8')

In [13]:
df_may = get_data_API(2017,5,1,31)
print len(df_may)

0


In [14]:
df_may.to_csv('./Data/Future/May2017.csv', encoding = 'utf-8')

In [15]:
df_jun = get_data_API(2017,6,1,30)
print len(df_jun)

0


In [16]:
df_jun.to_csv('./Data/Future/June2017.csv', encoding = 'utf-8')

In [17]:
df_jul = get_data_API(2017,7,1,31)
print len(df_jul)

0


In [18]:
df_jul.to_csv('./Data/Future/July2017.csv', encoding = 'utf-8')